### Homework
 - Finish scikit-learn practice competitions.

In [1]:
import os
import re
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import RandomizedSearchCV

In [2]:
folder_path = 'data/data2/scikit-learn-practice/'
ignore_file_pat = re.compile('(^\.[\w]+)')
file_names = os.listdir(folder_path)
file_names = list(filter(lambda x: ignore_file_pat.match(x) is None, file_names))
file_path = list(map(lambda x: os.path.join(folder_path, x), file_names))

print(file_path)

['data/data2/scikit-learn-practice/test.csv', 'data/data2/scikit-learn-practice/submission.csv', 'data/data2/scikit-learn-practice/trainLabels.csv', 'data/data2/scikit-learn-practice/train.csv']


In [3]:
x_train = pd.read_csv(file_path[3], header=None)
y_train = pd.read_csv(file_path[2], header=None)
x_test = pd.read_csv(file_path[0], header=None)

std_scaler = StandardScaler()
std_scaler.fit(x_train)
std_x_train = std_scaler.transform(x_train)
std_x_test = std_scaler.transform(x_test)

print('training data shape: {}'.format(x_train.shape))
print('testing data shape: {}'.format(x_test.shape))

training data shape: (1000, 40)
testing data shape: (9000, 40)


In [4]:
# Hyperparameter 
param_grid = {
    'n_estimators': np.linspace(400, 800, num=10).astype(np.int),
    'loss': ['deviance', 'exponential'],
    'learning_rate': np.linspace(0.1, 0.25, num=9),
    'min_samples_split': np.linspace(5, 20, num=16).astype(np.int),
    'max_depth': np.linspace(3, 10, 8).astype(np.int),
    'tol': np.linspace(0.0001, 0.001, num=10), 
    'max_features': ['auto', 'sqrt', 'log2']
}

gbm_clf = GradientBoostingClassifier()
RandSearch_gbm_clf = RandomizedSearchCV(estimator=gbm_clf,
                                        param_distributions=param_grid, cv=5)

result = RandSearch_gbm_clf.fit(std_x_train, y_train.values.ravel())

In [5]:
print('Beat Parameters: {}\nBest score: {:.2f} %'.format(result.best_params_, result.best_score_*100))

Beat Parameters: {'tol': 0.00039999999999999996, 'n_estimators': 444, 'min_samples_split': 8, 'max_features': 'log2', 'max_depth': 8, 'loss': 'exponential', 'learning_rate': 0.11875000000000001}
Best score: 88.50 %


In [6]:
y_pred = RandSearch_gbm_clf.predict(x_test)

In [7]:
output_filen_name = 'submission.csv'
output_file_path = os.path.join(folder_path, output_filen_name)

pd.DataFrame({'Id': list(range(1, x_test.shape[0] + 1)), 'Solution': y_pred}).to_csv(output_file_path, index=False)